# OpenAI Function Calling


**Notes**:
- LLM's don't always produce the same results. The results you see in this notebook may differ from the results you see in the video.
- Notebooks results are temporary. Download the notebooks to your local machine if you wish to save your results.

## outline (for this course)

- OpenAi function calling
- LangChain Expression Language (LCEL)
- OpenAI function calling in LangChain
- Tagging and extraction using OpenAI function calling
- Tools and Routing
- Conversational Agent

## Function calling

- OpenAI has fine-tuned the gpt-3.5 turbo-0613 and gpt-4-0613 models to :
  1. accept additional arguments through which users can pass in descriptions of functions.
  2. if it is relevant, return the nam of the function to use, along with a JSON object with teh appropriate input parameters.


#### did not really follow the tutorials, as the syntex are basically completely outdated/different

#### base codes: https://platform.openai.com/docs/guides/function-calling

In [3]:
# !pip install python-dotenv
# !pip install openai
# !pip install -U langchain langchain-openai

In [4]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']
LANGCHAIN_API_KEY = os.environ["LANGCHAIN_API_KEY"]
OPENAI_API_KEY = openai.api_key

In [5]:
import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [1]:
# define a function

# functions = [
#     {
#         "name": "get_current_weather",
#         "description": "Get the current weather in a given location",
#         "parameters": {
#             "type": "object",
#             "properties": {
#                 "location": {
#                     "type": "string",
#                     "description": "The city and state, e.g. San Francisco, CA",
#                 },
#                 "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
#             },
#             "required": ["location"],
#         },
#     }
# ]

In [6]:
# new syntex requires insert function into part of the tools

tools = [
    {
        "type": "function",
        "function":  {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
        }
    }
]

In [7]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston?"
    }
]

In [8]:
import openai

In [2]:
# an example of new syntex from: https://github.com/openai/openai-python
import os
from openai import OpenAI

client = OpenAI(
    # This cannot be omitted
    api_key= OPENAI_API_KEY
)

# model = ChatOpenAI(api_key= OPENAI_API_KEY)
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say this is a test",
        }
    ],
    model="gpt-3.5-turbo",
)

In [10]:
chat_completion

ChatCompletion(id='chatcmpl-A9iSGhq2FALs2AudZXCj6ce8anvI5', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='This is a test.', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1726879200, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=5, prompt_tokens=12, total_tokens=17, completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0)))

In [11]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_delivery_date",
            "description": "Get the delivery date for a customer's order. Call this whenever you need to know the delivery date, for example when a customer asks 'Where is my package'",
            "parameters": {
                "type": "object",
                "properties": {
                    "order_id": {
                        "type": "string",
                        "description": "The customer's order ID.",
                    },
                },
                "required": ["order_id"],
                "additionalProperties": False,
            },
        }
    }
]

messages = [
    {"role": "system", "content": "You are a helpful customer support assistant. Use the supplied tools to assist the user."},
    {"role": "user", "content": "Hi, what is the weather like in Boston?"}
]

response = openai.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
    # functions = functions,
    # function_call and functions go together
    # function_call="auto",
)

In [12]:
response

ChatCompletion(id='chatcmpl-A9iShD6QfaJocmN5F2D3xz8kA1DCE', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="I currently don't have access to real-time weather information. You may want to check a reliable weather website or use a weather app for the most accurate and up-to-date details on the weather in Boston.", refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1726879227, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_3537616b13', usage=CompletionUsage(completion_tokens=41, prompt_tokens=104, total_tokens=145, completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0)))

In [13]:
# source: https://platform.openai.com/docs/guides/chat-completions/getting-started
# https://platform.openai.com/docs/api-reference/chat/streaming

from openai import OpenAI
client = OpenAI(
    api_key= OPENAI_API_KEY
)

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages= messages,
  # new syntex
  tools=tools,
  # function_call="auto"
)
response

ChatCompletion(id='chatcmpl-A9iTNPnp9bFhbjXE42xmtu5nB6FYY', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="I'm unable to provide real-time weather information. However, you can easily check the weather for Boston on various weather websites or apps like the Weather Channel, AccuWeather, or your smartphone’s weather app.", refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1726879269, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_1bb46167f9', usage=CompletionUsage(completion_tokens=42, prompt_tokens=104, total_tokens=146, completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0)))

In [17]:
# source: https://platform.openai.com/docs/guides/chat-completions/getting-started
# https://platform.openai.com/docs/api-reference/chat/streaming

from openai import OpenAI
client = OpenAI(
    api_key= OPENAI_API_KEY
)

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Who won the world series in 2020?"},
    {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
    {"role": "user", "content": "Where was it played?"}
  ]
)

response

ChatCompletion(id='chatcmpl-A9iUhb6VmUPRs1zaqFEhCoxTbopgh', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The 2020 World Series was played at Globe Life Field in Arlington, Texas. This was a neutral site due to the COVID-19 pandemic.', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1726879351, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_1bb46167f9', usage=CompletionUsage(completion_tokens=30, prompt_tokens=53, total_tokens=83, completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0)))

In [27]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_delivery_date",
            "description": "Get the delivery date for a customer's order. Call this whenever you need to know the delivery date, for example when a customer asks 'Where is my package'",
            "parameters": {
                "type": "object",
                "properties": {
                    "order_id": {
                        "type": "string",
                        "description": "The customer's order ID."
                    }
                },
                "required": ["order_id"],
                "additionalProperties": False
            }
        }
    }
]

messages = []
messages.append({"role": "system", "content": "You are a helpful customer support assistant. Use the supplied tools to assist the user."})
messages.append({"role": "user", "content": "Hi, can you tell me the delivery date for my order?"})
# # // highlight-start
# does not work if you keep adding
# messages.append({"role": "assistant", "content": "Hi there! I can help with that. Can you please provide your order ID?"})
# messages.append({"role": "user", "content": "i think it is order_12345"})
# // highlight-end

response = client.chat.completions.create(
    model='gpt-4o',
    messages=messages,
    tools=tools
)

In [28]:
response

ChatCompletion(id='chatcmpl-A9iYpO9WGjOZoowomrKhugG11jy2d', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Sure, I'd be happy to help with that. Could you please provide me with your order ID?", refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1726879607, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_a2ff031fb5', usage=CompletionUsage(completion_tokens=21, prompt_tokens=107, total_tokens=128, completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0)))

In [30]:
# Simulate the order_id and delivery_date
from datetime import datetime
order_id = "order_12345"
delivery_date = datetime.now()

# simulate the tool call response
response = {
    "choices": [
        {
            "message": {
                "role": "assistant",
                "tool_calls": [
                    {
                        "id": "call_62136354",
                        "type": "function",
                        "function": {
                            "arguments": "{'order_id': 'order_12345'}",
                            "name": "get_delivery_date"
                        }
                    }
                ]
            }
        }
    ]
}

function_call_result_message = {
    "role": "tool",
    "content": json.dumps({
        "order_id": order_id,
        "delivery_date": delivery_date.strftime('%Y-%m-%d %H:%M:%S')
    }),
    "tool_call_id": response['choices'][0]['message']['tool_calls'][0]['id']
}

# chat completion payload
completion_payload = {
    "model": "gpt-4o",
    "messages": [
        {"role": "system", "content": "You are a helpful customer support assistant. Use the supplied tools to assist the user."},
        {"role": "user", "content": "Hi, can you tell me the delivery date for my order?"},
        {"role": "assistant", "content": "Hi there! I can help with that. Can you please provide your order ID?"},
        {"role": "user", "content": "i think it is order_12345"},
        response['choices'][0]['message'],
        function_call_result_message
    ]
}

# call the OpenAI API's chat completions endpoint to send the tool call result back to the model
response = openai.chat.completions.create(
    model=completion_payload["model"],
    messages=completion_payload["messages"]
)


In [31]:
response

ChatCompletion(id='chatcmpl-A9iZqyYfc2wZYQ7GnH9DtZJuslqRW', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Your order (ID: order_12345) is scheduled to be delivered on **September 21, 2024**. If you have any other questions or need further assistance, feel free to ask!', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1726879670, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_e375328146', usage=CompletionUsage(completion_tokens=42, prompt_tokens=131, total_tokens=173, completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0)))